In [34]:
## read data
import os
import shutil
import json
import matplotlib.pyplot as plt 
import re
import numpy as np
import pandas as pd 
import scipy.stats as stats
import statsmodels.api as sm
from datetime import datetime
from statsmodels.formula.api import ols
print(os.getcwd())
os.chdir("Desktop/2021fall/628/group3/milktea")
print(os.getcwd())

/Users/qiluzhou/Desktop/2021fall/628/group3/milktea


FileNotFoundError: [Errno 2] No such file or directory: 'Desktop/2021fall/628/group3/milktea'

In [2]:
# #read reviews
# reviews=[]
# with open("bubble_tea_review.json",'r',encoding="UTF-8") as f:
#     for i in f:
#         reviews.append(json.loads(i))
# print(reviews[:2])

In [3]:
#read business
business=[]
with open("bubble_tea_business.json",'r',encoding="UTF-8") as f:
    for i in f:
        business.append(json.loads(i))
print(business[:2])

[{'business_id': 'n-_p1jwGCbZT9YaTbugsdw', 'name': 'Vivi bubble Tea', 'address': '1111 E Colonial Dr, Ste 1111', 'city': 'Orlando', 'state': 'FL', 'postal_code': '32803', 'latitude': 28.5534641199, 'longitude': -81.3649773955, 'stars': 4.0, 'review_count': 197, 'is_open': 1, 'attributes': {'GoodForKids': 'True', 'WheelchairAccessible': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': True, 'valet': False}", 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", 'DogsAllowed': 'False', 'NoiseLevel': "u'average'", 'RestaurantsTakeOut': 'True', 'RestaurantsReservations': 'True', 'Caters': 'False', 'RestaurantsPriceRange2': '1', 'BikeParking': 'True', 'BusinessAcceptsCreditCards': 'True'}, 'categories': 'Coffee & Tea, Food, Hot Dogs, Restaurants, Bubble Tea', 'hours': {'Monday': '11:0-22:30', 'Tuesday': '11:0-22:30', 'Wednesday': '11:0-22:30', 'Thursday': '11:0-22:30', 'Friday': 

In [7]:
import pandas as pd 
business=pd.DataFrame(business)
print(business.head())
print(len(business)) #there are 779 milktea shops
business.isnull().sum() #24 na's in attributes and 78 na's in hours
business.dropna(axis=0,how='any',inplace=True)

              business_id                     name  \
0  n-_p1jwGCbZT9YaTbugsdw          Vivi bubble Tea   
1  3thW8u8cgdCLKgYhKMhXBw             A Thai Basil   
2  A0F6H8OO3qYAvI2L3OCp1g    Dragon Ball Tea House   
3  rIzmyKEsPZsyhHph5n0Nag               Chewy Boba   
4  ALP7rrHJxXIHyzMMPGQ1vw  LittleMoon Cafe and Tea   

                        address         city state postal_code   latitude  \
0  1111 E Colonial Dr, Ste 1111      Orlando    FL       32803  28.553464   
1            SW 10th & Alder St     Portland    OR       97205  45.520536   
2     1007 W King Edward Avenue    Vancouver    BC     V6H 1Z3  49.249492   
3            106 S Semoran Blvd  Winter Park    FL       32792  28.596584   
4                   2899 N High     Columbus    OH       43202  40.021372   

    longitude  stars  review_count  is_open  \
0  -81.364977    4.0           197        1   
1 -122.681604    3.0            41        0   
2 -123.127926    4.0           265        1   
3  -81.307875    4.0    

In [5]:
# #category
# all_cate="".join(business['categories'])
# all_cate=all_cate.split(",")
# counts={}
# for word in all_cate:
#     counts[word]=counts.get(word,0)+1
# items=list(counts.items())
# items.sort(key=lambda x:x[1], reverse=True)
# for i in range(20):
#     word,count=items[i]
#     print("{0:<10}{1:>5}".format(word,count))

analyzing all the categories of bubble tea shop, we can find the top 20 words in categories above. 
1. bubble tea shops also offer food, and some of them are restaurants
2. they sell many kinds of drinks, for instance coffee&tea, juice&smoothies
3. they also sell desserts, for example ice cream and yogurt
4. Chinese, Taiwanese, Vietnamese are the top three regional labels in categories, and some of the japanese food restaurants sell bubble tea too

In [8]:
#attributes
attributes=business['attributes']
type(attributes)
#attributes=list(filter(None,list(attributes.to_dict().values())))
attributes=list(attributes.to_dict().values())
att=[]
for i in attributes:
    att.append(i.keys())
all_att=str(re.split(",|dict_keys",str(att)))
all_att=all_att.replace("])",'').replace("([",'').replace("[",'')
all_att=re.split(',', all_att)
# select the top 20 attributes 
counts={}
top_att=[]
for word in all_att:
    counts[word]=counts.get(word,0)+1
items=list(counts.items())
items.sort(key=lambda x:x[1], reverse=True)
for i in range(21):
    top_att.append(items[i][0])
    word,count=items[i]
    print("{0:<10}{1:>5}".format(word,count))

 ' '        695
 " 'RestaurantsTakeOut'"  589
 " 'BusinessParking'"  560
 " 'RestaurantsDelivery'"  538
 " 'WiFi'"  501
 " 'Caters'"  480
 " 'BikeParking'"  471
 " 'RestaurantsPriceRange2'"  461
 " 'BusinessAcceptsCreditCards'"  424
 " 'OutdoorSeating'"  411
 " 'WheelchairAccessible'"  368
 " 'DogsAllowed'"  319
 " 'GoodForKids'"  299
 " 'NoiseLevel'"  294
 " 'Alcohol'"  262
 " 'HasTV'"  257
 " 'RestaurantsReservations'"  255
 " 'Ambience'"  243
 " 'RestaurantsGoodForGroups'"  223
 " 'GoodForMeal'"  218
 " 'RestaurantsAttire'"  195


Top 20 attributes is above. So next we will only focus on these attributes to analyze their relation to stars. 
1. In the top 20 attributes, we could see that for most of the bubble tea shops, a common feature is that they offer take out or delivery. 
2. Also parking to cars and bikes are also important. 
3. The price of them are mostly in range 2, which is an affordable price for most of the customers and as a drink or dessert. 
4. For payment, they mostly accept credit cards. 
5. Most of them have wifi so that people could work there.

In [9]:
top20=re.findall('[a-zA-z]+',"".join(top_att[1:]))

In [10]:
def find_att(x,y=attributes):
    items=[]
    for i in y:
        items.append(i.get(x))
    return(items)

In [11]:
# aggregate id,stars and attributes
df=pd.DataFrame()
df['id']=list(business['business_id'])
df['stars']=list(business['stars'])
for i in top20:
    df[i]=find_att(i)

In [12]:
print(df.head())
np.mean(df['stars'])#mean stars is 3.84, which is relatively high

                       id  stars RestaurantsTakeOut  \
0  n-_p1jwGCbZT9YaTbugsdw    4.0               True   
1  A0F6H8OO3qYAvI2L3OCp1g    4.0               True   
2  rIzmyKEsPZsyhHph5n0Nag    4.0               True   
3  ALP7rrHJxXIHyzMMPGQ1vw    4.5               True   
4  DrQuKhjlSbamzC9F0lXOzQ    3.5               True   

                                     BusinessParking RestaurantsDelivery  \
0  {'garage': False, 'street': True, 'validated':...                None   
1  {'garage': False, 'street': True, 'validated':...                None   
2  {'garage': False, 'street': False, 'validated'...                None   
3  {'garage': False, 'street': False, 'validated'...                True   
4  {'garage': False, 'street': False, 'validated'...                True   

      WiFi Caters BikeParking RestaurantsPriceRange  \
0     None  False        True                  None   
1  u'free'  False        True                  None   
2   'free'  False        True                  

3.8426724137931036

In [13]:
df['WiFi']=df['WiFi'].replace("u'free'","'free'")
df['WiFi']=df['WiFi'].replace("u'paid'","'paid'")
df['WiFi']=df['WiFi'].replace("u'no'","'no'")
df['NoiseLevel']=df['NoiseLevel'].replace("u'average'","'average'")
df['NoiseLevel']=df['NoiseLevel'].replace("u'loud'","'loud'")
df['NoiseLevel']=df['NoiseLevel'].replace("u'quiet'","'quiet'")
df['NoiseLevel']=df['NoiseLevel'].replace("u'very_loud'","'very_loud'")

In [91]:
# group=df.groupby('RestaurantsTakeOut')
# g1=group.mean()
# g1
# group=df.groupby('RestaurantsTakeOut')
# g_count=group.count()
# #g_count['RestaurantsTakeOut']
# # function of plotting attributes and stars
# # def star_plot(name):
# #     #average
# #     group=df.groupby(name)
# #     g1=group.mean()
# #     print(g1)
# #     #count
# #     group=df.groupby(name)
# #     g_count=group.count()
# #     print(g_count['stars'])
# #     #plot
# #     plt.bar(g1.index,list(g1['stars']))
# #     plt.ylabel('stars')
# #     plt.xlabel(name)
# #     plt.title(name)
# #     plt.ylim(0,5)
# #     plt.show()
# #attribute plots
# # for i in top20:
# #     star_plot(i)

1. Use ANOVA to find significant attributes of all the business data
2. Write a function to use location and name to find category, attributes, stars of one shop（in R？）
3. Separate all attributes in one data，and download csv
4. Compare the attributes of shop and significant attributes

In [30]:
# find all attributes
attributes=business['attributes']
attributes=list(attributes.to_dict().values())
att=[]
for i in attributes:
    att.append(i.keys())
all_att=str(re.split(",|dict_keys",str(att)))
all_att=all_att.replace("])",'').replace("([",'').replace("[",'')
all_att=re.split(',', all_att)
counts={}
att_list=[]
for word in all_att:
    counts[word]=counts.get(word,0)+1
items=list(counts.items())
items.sort(key=lambda x:x[1], reverse=True)
for i in range(len(items)):
    att_list.append(items[i][0])
    word,count=items[i]
    #print("{0:<10}{1:>5}".format(word,count))
att_list=re.findall('[a-zA-z]+',"".join(att_list[0:]))
att_list=att_list[:-2]

In [125]:
# aggregate id,stars and attributes
total=pd.DataFrame()
total['business_id']=list(business['business_id'])
total['stars']=list(business['stars'])
for i in att_list:
    total[i]=find_att(i)

In [145]:
business_data=pd.merge(business,total,on=['business_id','stars'])
business_data.drop(['attributes'],axis=1,inplace=True)
business_data=pd.concat([business_data,business_data['hours'].apply(pd.Series)],axis=1).drop('hours',axis=1)

In [152]:
business_data.fillna('None',inplace=True)

In [153]:
business_data

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,CoatCheck,BYOBCorkage,AcceptsInsurance,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,n-_p1jwGCbZT9YaTbugsdw,Vivi bubble Tea,"1111 E Colonial Dr, Ste 1111",Orlando,FL,32803,28.553464,-81.364977,4.0,197,...,None,None,None,11:0-22:30,11:0-22:30,11:0-22:30,11:0-22:30,11:0-23:0,11:0-23:0,11:0-23:0
1,A0F6H8OO3qYAvI2L3OCp1g,Dragon Ball Tea House,1007 W King Edward Avenue,Vancouver,BC,V6H 1Z3,49.249492,-123.127926,4.0,265,...,None,None,None,14:0-23:0,14:0-23:0,14:0-23:0,14:0-23:0,14:0-0:0,14:0-0:0,14:0-23:0
2,rIzmyKEsPZsyhHph5n0Nag,Chewy Boba,106 S Semoran Blvd,Winter Park,FL,32792,28.596584,-81.307875,4.0,51,...,None,None,None,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,12:0-21:0
3,ALP7rrHJxXIHyzMMPGQ1vw,LittleMoon Cafe and Tea,2899 N High,Columbus,OH,43202,40.021372,-83.013519,4.5,17,...,None,None,None,0:0-0:0,13:0-20:30,None,13:0-20:30,13:0-20:30,13:0-20:30,13:0-20:30
4,DrQuKhjlSbamzC9F0lXOzQ,Cloud 9,14201 SE Mill Plain Blvd,Vancouver,WA,98684,45.617417,-122.526081,3.5,76,...,None,None,None,0:0-0:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,757zI3bNk4CZ9cql3T6rbQ,Pho 98°C,92 N Main St,Randolph,MA,02368,42.164487,-71.042618,4.5,232,...,False,None,None,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0,10:0-22:0
692,LBpKl-sl0Qe18oAdoiDaCQ,Tiki Tea,4936 NE 16th,Portland,OR,97211,45.558785,-122.648560,4.5,187,...,None,None,None,0:0-0:0,None,12:0-18:30,12:0-18:30,12:0-19:0,12:0-19:0,12:0-19:0
693,aesuEZoyNuctrsKzo-OIUA,EL Yucateco Food Cart,13110 NE Sandy Blvd,Portland,OR,97230,45.555263,-122.528067,4.5,65,...,None,None,None,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-20:0,11:0-17:0
694,W0nk_v3uGiP7CoiW919oeg,YiFang Taiwan Fruit Tea,3612 Kingsway,Vancouver,BC,V5R 5M2,49.232031,-123.026117,3.5,23,...,None,None,None,13:0-22:0,13:0-22:0,13:0-22:0,13:0-22:0,13:0-23:0,11:0-23:0,11:0-22:0


In [162]:
def openhours(x):
    day=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
    hour=[]
    for day in day:
        if x[day]!='None':
            a1=datetime.strptime(x[day].split("-")[1], "%H:%M")
            a0=datetime.strptime(x[day].split("-")[0], "%H:%M")
            hour.append((a1-a0).seconds/3600)
    return(sum(hour))

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [163]:
n=len(business_data['Monday'])
weekhour=[]
for i in range(n):
    weekhour.append(openhours(business_data.iloc[i,:]))
business_data['weekhour']=weekhour

['14:0', '23:0']

In [216]:
#business_data.to_csv("total.csv")